In [ ]:
import pickle
import pandas as pd
import itertools
from collections import Counter
import numpy as np
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from gensim.models import word2vec
from sklearn.linear_model import LogisticRegression
import os
import string
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
data_path = "/content/drive/MyDrive/cse 291 datamining challenge/dm_files/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df_train = pd.read_csv(data_path + "train.csv")
df_test = pd.read_csv(data_path + "test.csv")
df_train["text"] = df_train["review"]
df_test["text"] = df_test["review"]

In [ ]:
df_train.head()

,id,attributes.HappyHour,attributes.Ambience,hours.Tuesday,postal_code,attributes.AgesAllowed,attributes.GoodForDancing,attributes.OutdoorSeating,hours.Saturday,attributes.Corkage,...,attributes.RestaurantsDelivery,attributes.DietaryRestrictions,attributes.BusinessAcceptsBitcoin,address,attributes.GoodForKids,attributes.GoodForMeal,hours,label,review,text
0,0,b'True',"b""{'romantic': False, 'intimate': False, 'clas...",b'15:0-2:0',b'44107',NaN,NaN,b'False',b'11:30-2:0',NaN,...,b'False',NaN,NaN,b'17800 Detroit Ave',b'False',"b""{'dessert': False, 'latenight': False, 'lunc...","{'Monday': '16:0-2:0', 'Tuesday': '15:0-2:0', ...",american (traditional),"So, we stopped here on our way to the Side Que...","So, we stopped here on our way to the Side Que..."
1,1,NaN,"b""{'romantic': False, 'intimate': False, 'tour...",b'11:0-21:0',b'85042',NaN,NaN,b'True',b'11:0-20:30',NaN,...,b'False',NaN,b'False',"b'2160 E Baseline Rd, Ste 128'",b'True',"b""{'dessert': False, 'latenight': False, 'lunc...","{'Monday': '11:0-21:0', 'Tuesday': '11:0-21:0'...",american (new),This is our go-to healthy spot! The food is al...,This is our go-to healthy spot! The food is al...
2,2,NaN,NaN,b'11:0-21:0',b'M4M 3G6',NaN,NaN,NaN,b'11:0-21:0',NaN,...,NaN,NaN,NaN,b'1000 Gerrard St E',NaN,NaN,"{'Monday': '11:0-21:0', 'Tuesday': '11:0-21:0'...",mexican,Food court meal at Gerrard Square. It's been ...,Food court meal at Gerrard Square. It's been ...
3,3,NaN,"b""{'romantic': False, 'intimate': False, 'clas...",NaN,b'89146',NaN,NaN,b'False',NaN,NaN,...,b'True',NaN,NaN,b'6700 W Charleston Blvd',b'True',NaN,NaN,mexican,"Located on Rainbow/Charleston, this small fami...","Located on Rainbow/Charleston, this small fami..."
4,4,NaN,"b""{'romantic': False, 'intimate': False, 'tour...",NaN,b'44133',NaN,NaN,b'False',NaN,NaN,...,b'False',NaN,NaN,b'5630 Wallings Rd',b'True',"b""{'dessert': False, 'latenight': False, 'lunc...",NaN,chinese,No frills Chinese takeout joint which serves u...,No frills Chinese takeout joint which serves u...


In [ ]:
df_train.head()['postal_code']

0      b'44107'
1      b'85042'
2    b'M4M 3G6'
3      b'89146'
4      b'44133'
Name: postal_code, dtype: object

In [ ]:
def preprocess_df(df):
    stop_words = set(stopwords.words('english'))
    stop_words.add('would')
    translator = str.maketrans(string.punctuation, ' ' * len(string.punctuation))
    preprocessed_sentences = []
    for i, row in df.iterrows():
        sent = row["text"]
        sent_nopuncts = sent.translate(translator)
        words_list = sent_nopuncts.strip().split()
        filtered_words = [word for word in words_list if word not in stop_words and len(word) != 1]
        preprocessed_sentences.append(" ".join(filtered_words))
    df["text"] = preprocessed_sentences
    return df

In [ ]:
df_train = preprocess_df(df_train)
df_test = preprocess_df(df_test)

In [ ]:
df_train.head()

,id,attributes.HappyHour,attributes.Ambience,hours.Tuesday,postal_code,attributes.AgesAllowed,attributes.GoodForDancing,attributes.OutdoorSeating,hours.Saturday,attributes.Corkage,...,attributes.RestaurantsDelivery,attributes.DietaryRestrictions,attributes.BusinessAcceptsBitcoin,address,attributes.GoodForKids,attributes.GoodForMeal,hours,label,review,text
0,0,b'True',"b""{'romantic': False, 'intimate': False, 'clas...",b'15:0-2:0',b'44107',NaN,NaN,b'False',b'11:30-2:0',NaN,...,b'False',NaN,NaN,b'17800 Detroit Ave',b'False',"b""{'dessert': False, 'latenight': False, 'lunc...","{'Monday': '16:0-2:0', 'Tuesday': '15:0-2:0', ...",american (traditional),"So, we stopped here on our way to the Side Que...",So stopped way Side Quest street nWe know expe...
1,1,NaN,"b""{'romantic': False, 'intimate': False, 'tour...",b'11:0-21:0',b'85042',NaN,NaN,b'True',b'11:0-20:30',NaN,...,b'False',NaN,b'False',"b'2160 E Baseline Rd, Ste 128'",b'True',"b""{'dessert': False, 'latenight': False, 'lunc...","{'Monday': '11:0-21:0', 'Tuesday': '11:0-21:0'...",american (new),This is our go-to healthy spot! The food is al...,This go healthy spot The food always fresh del...
2,2,NaN,NaN,b'11:0-21:0',b'M4M 3G6',NaN,NaN,NaN,b'11:0-21:0',NaN,...,NaN,NaN,NaN,b'1000 Gerrard St E',NaN,NaN,"{'Monday': '11:0-21:0', 'Tuesday': '11:0-21:0'...",mexican,Food court meal at Gerrard Square. It's been ...,Food court meal Gerrard Square It since nachos...
3,3,NaN,"b""{'romantic': False, 'intimate': False, 'clas...",NaN,b'89146',NaN,NaN,b'False',NaN,NaN,...,b'True',NaN,NaN,b'6700 W Charleston Blvd',b'True',NaN,NaN,mexican,"Located on Rainbow/Charleston, this small fami...",Located Rainbow Charleston small family owned ...
4,4,NaN,"b""{'romantic': False, 'intimate': False, 'tour...",NaN,b'44133',NaN,NaN,b'False',NaN,NaN,...,b'False',NaN,NaN,b'5630 Wallings Rd',b'True',"b""{'dessert': False, 'latenight': False, 'lunc...",NaN,chinese,No frills Chinese takeout joint which serves u...,No frills Chinese takeout joint serves best ar...


In [ ]:
label_list=df_train["label"].unique()

In [ ]:
label_list

array(['american (traditional)', 'american (new)', 'mexican', 'chinese',
       'mediterranean', 'italian', 'thai', 'asian fusion',
       'canadian (new)', 'japanese'], dtype=object)

In [ ]:
df_train["labels"]=df_train["label"]

In [ ]:
for index, row in df_train.iterrows():
  df_train.at[index,"labels"]=np.where(label_list==df_train.iloc[index]["labels"])[0][0]

In [ ]:
  df_train["labels"].head

<bound method NDFrame.head of 0        0
1        1
2        2
3        2
4        3
        ..
13139    2
13140    5
13141    2
13142    9
13143    3
Name: labels, Length: 13144, dtype: object>

In [ ]:
type(df_train.iloc[0]['attributes.Ambience'])

str

In [ ]:
from tqdm.auto import tqdm

In [ ]:
# def preprocess(df):

#   for index, row in tqdm(df.iterrows()):
#     if not pd.isnull(df.iloc[index]["attributes.HappyHour"]):
#       df_train.at[index,"train_text"]+=" Happy Hour: "+df.iloc[index]["attributes.HappyHour"].replace("b'","").replace("'","")
#     if not pd.isnull(df.iloc[index]["attributes.Ambience"]):
#       df_train.at[index,"train_text"]+=" Ambience review: "+df.iloc[index]["attributes.Ambience"].replace("b'","").replace("'","")
#     if not pd.isnull(df.iloc[index]["hours.Tuesday"]):
#       df_train.at[index,"train_text"]+=" hours on Tuesday: "+df.iloc[index]["hours.Tuesday"].replace("b'","").replace("'","")
#     if not pd.isnull(df.iloc[index]["postal_code"]):
#       df_train.at[index,"train_text"]+=" postal code: "+df.iloc[index]["postal_code"].replace("b'","").replace("'","")
#     if not pd.isnull(df.iloc[index]["attributes.AgesAllowed"]):
#       df_train.at[index,"train_text"]+=" Age limitation: "+str(df.iloc[index]["attributes.AgesAllowed"]).replace("b'","").replace("'","")
#     if not pd.isnull(df.iloc[index]["attributes.GoodForDancing"]):
#       df_train.at[index,"train_text"]+=" Is this place good for dance: "+df.iloc[index]["attributes.GoodForDancing"].replace("b'","").replace("'","")
#     if not pd.isnull(df.iloc[index]["attributes.RestaurantsDelivery"]):
#       df_train.at[index,"train_text"]+=" Delivery avaliable: "+df.iloc[index]["attributes.RestaurantsDelivery"].replace("b'","").replace("'","")
#     if not pd.isnull(df.iloc[index]["attributes.GoodForKids"]):
#       df_train.at[index,"train_text"]+=" Good For Kids: "+df.iloc[index]["attributes.GoodForKids"].replace("b'","").replace("'","")
#     if not pd.isnull(df.iloc[index]["attributes.GoodForMeal"]):
#       df_train.at[index,"train_text"]+=" Meal avaliable:"+df.iloc[index]["attributes.GoodForMeal"].replace("b'","").replace("'","")
#     if not pd.isnull(df.iloc[index]["hours"]):
#       df_train.at[index,"train_text"]+=" Opening Hours:"+df.iloc[index]["hours"].replace("b'","").replace("'","")
#     if not pd.isnull(df.iloc[index]["attributes.OutdoorSeating"]):
#       df_train.at[index,"train_text"]+=" outdoor seat:"+df.iloc[index]["attributes.OutdoorSeating"].replace("b'","").replace("'","")
#     if not pd.isnull(df.iloc[index]["hours.Saturday"]):
#       df_train.at[index,"train_text"]+="hours on Saturday:"+df.iloc[index]["hours.Saturday"].replace("b'","").replace("'","")
#     if not pd.isnull(df.iloc[index]["attributes.Corkage"]):
#       df_train.at[index,"train_text"]+=" Corkage avaliable:"+df.iloc[index]["attributes.Corkage"].replace("b'","").replace("'","")
#     if not pd.isnull(df.iloc[index]["attributes.DietaryRestrictions"]):
#       df_train.at[index,"train_text"]+=" Dietary Restrictions:"+df.iloc[index]["attributes.DietaryRestrictions"].replace("b'","").replace("'","")
#     if not pd.isnull(df.iloc[index]["attributes.BusinessAcceptsBitcoin"]):
#       df_train.at[index,"train_text"]+=" Can customer use bitcoin:"+df.iloc[index]["attributes.BusinessAcceptsBitcoin"].replace("b'","").replace("'","")
#     if not pd.isnull(df.iloc[index]["address"]):
#       df_train.at[index,"train_text"]+=" address:"+df.iloc[index]["address"].replace("b'","").replace("'","")
#     df["train_text"]="customer review: "+df_train["review"]
#   return df

In [ ]:
def preprocess(df):
  df["train_text"]=""
  for index, row in tqdm(df.iterrows()):
    df.at[index,"train_text"]+=" Happy Hour: "+str(df.iloc[index]["attributes.HappyHour"]).replace("b'","").replace("'","").replace("\'","")
    df.at[index,"train_text"]+=" Ambience review: "+str(df.iloc[index]["attributes.Ambience"]).replace('b"',"").replace('"',"").replace("\'","")
    df.at[index,"train_text"]+=" hours on Tuesday: "+str(df.iloc[index]["hours.Tuesday"]).replace("b'","").replace("'","").replace("\'","")
    df.at[index,"train_text"]+=" postal code: "+str(df.iloc[index]["postal_code"]).replace("b'","").replace("'","").replace("\'","")
    df.at[index,"train_text"]+=" Age limitation: "+str(df.iloc[index]["attributes.AgesAllowed"]).replace("b'","").replace("'","").replace("\'","")
    df.at[index,"train_text"]+=" Is this place good for dance: "+str(df.iloc[index]["attributes.GoodForDancing"]).replace("b'","").replace("'","").replace("\'","")
    df.at[index,"train_text"]+=" Delivery avaliable: "+str(df.iloc[index]["attributes.RestaurantsDelivery"]).replace("b'","").replace("'","").replace("\'","")
    df.at[index,"train_text"]+=" Good For Kids: "+str(df.iloc[index]["attributes.GoodForKids"]).replace("b'","").replace("'","").replace("\'","")
    df.at[index,"train_text"]+=" Meal avaliable: "+str(df.iloc[index]["attributes.GoodForMeal"]).replace('b"',"").replace('"',"").replace("\'","")
    df.at[index,"train_text"]+=" Opening Hours: "+str(df.iloc[index]["hours"]).replace("b'","").replace("'","").replace("\'","")
    df.at[index,"train_text"]+=" outdoor seat: "+str(df.iloc[index]["attributes.OutdoorSeating"]).replace("b'","").replace("'","").replace("\'","")
    df.at[index,"train_text"]+="hours on Saturday: "+str(df.iloc[index]["hours.Saturday"]).replace("b'","").replace("'","").replace("\'","")
    df.at[index,"train_text"]+=" Corkage avaliable: "+str(df.iloc[index]["attributes.Corkage"]).replace("b'","").replace("'","").replace("\'","")
    df.at[index,"train_text"]+=" Dietary Restrictions: "+str(df.iloc[index]["attributes.DietaryRestrictions"]).replace("b'","").replace("'","").replace("\'","")
    df.at[index,"train_text"]+=" Can customer use bitcoin: "+str(df.iloc[index]["attributes.BusinessAcceptsBitcoin"]).replace("b'","").replace("'","").replace("\'","")
    df.at[index,"train_text"]+=" address: "+str(df.iloc[index]["address"]).replace("b'","").replace("'","").replace("\'","")
    df.at[index,"train_text"]+=" customer review: "+str(df.iloc[index]["text"])
  return df

In [ ]:
df_train=preprocess(df_train)

0it [00:00, ?it/s]

In [ ]:
df_train['train_text'][0]

' Happy Hour: True Ambience review: {romantic: False, intimate: False, classy: False, hipster: False, divey: False, touristy: False, trendy: False, upscale: False, casual: True} hours on Tuesday: 15:0-2:0 postal code: 44107 Age limitation: nan Is this place good for dance: nan Delivery avaliable: False Good For Kids: False Meal avaliable: {dessert: False, latenight: False, lunch: True, dinner: True, brunch: False, breakfast: False} Opening Hours: {Monday: 16:0-2:0, Tuesday: 15:0-2:0, Wednesday: 15:0-2:0, Thursday: 11:30-2:0, Friday: 11:30-2:0, Saturday: 11:30-2:0} outdoor seat: Falsehours on Saturday: 11:30-2:0 Corkage avaliable: nan Dietary Restrictions: nan Can customer use bitcoin: nan address: 17800 Detroit Ave customer review: So stopped way Side Quest street nWe know expect came taco night nI glad nMostly Rush Inn bar ish place plenty back side seating booths The place dark rich wood tone patrons decent folks nThe tacos EXCEPTIONAL selection quite good sorts stuff intend try soon

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(df_train['train_text'], df_train['labels'], test_size=0.3, random_state=52)

In [ ]:
len(X_valid)

3944

In [ ]:
import torch
assert torch.cuda.is_available()
device = torch.device("cuda")

In [ ]:
pip  install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 89.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 121.1 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AdamW,BertForSequenceClassification

In [ ]:
tokenizer= AutoTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
X_train=tokenizer(list(X_train),truncation=True, padding=True, max_length=512,return_tensors='pt')
y_train=list(y_train)

In [ ]:
X_valid=tokenizer(list(X_valid),truncation=True, padding=True, max_length=512,return_tensors='pt')
y_valid=list(y_valid)

In [ ]:
class MyData(torch.utils.data.Dataset):
    def __init__(self, token):
        self.token = token
    def __getitem__(self, id):
        return {key: val[id] for key, val in self.token.items()}
    def __len__(self):
        return len(self.token.input_ids)

In [ ]:
bat_size=16

In [ ]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(MyData(X_train) , batch_size=bat_size)
valid_dataloader = DataLoader(MyData(X_valid) , batch_size=bat_size)

In [ ]:
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels = len(label_list),
    output_attentions = False,
    output_hidden_states = False,
)
model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
optimizer = AdamW(model.parameters(),
                  lr = 1e-6, # args.learning_rate - default is 5e-5
                  eps = 1e-8, # args.adam_epsilon  - default is 1e-8
                  weight_decay=0.01,
                  correct_bias=True
                )
epochs = 300

In [ ]:
model.load_state_dict(torch.load("/content/drive/MyDrive/cse 291 datamining challenge/dm_files/my_model"))

<All keys matched successfully>

In [ ]:
train_lost_epoch=[]
valid_lost_epoch=[]
valid_accuracy_list=[]
for epoch in tqdm(range(epochs),desc='epoch'):
    model.train()
    loss_list=[]
    i=0
    for batch in tqdm(train_dataloader,desc='trainset',leave=False):
        model.zero_grad()
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        target=torch.tensor(y_train[i:i+bat_size]).to(device)
        i+=bat_size
        outputs = model(input_ids,
                            token_type_ids,
                            attention_mask,
                            labels=target)

        loss = outputs.loss
        logits = outputs.logits
        loss.backward()
        optimizer.step()
        loss_list.append(loss.item())
    train_lost_epoch.append(np.mean(loss_list))
    model.eval()
    loss_list=[]
    i=0
    num_correct=0
    for batch in tqdm(valid_dataloader,desc='validset',leave=False):
      with torch.no_grad():
        input_ids = batch['input_ids'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        target=torch.tensor(y_valid[i:i+bat_size]).to(device)
        i+=bat_size
        outputs = model(input_ids,
                            token_type_ids,
                            attention_mask,
                            labels=target)
        loss = outputs.loss
        logits = outputs.logits
        loss_list.append(loss.item())
        pred_flat = np.argmax(logits.detach().cpu().numpy(), axis=1).flatten()
        labels_flat = target.to('cpu').numpy().flatten()
        num_correct += np.sum(pred_flat == labels_flat)
    valid_accuracy=num_correct/len(y_valid)
    if  epoch==0:
      torch.save(model.state_dict(), "/content/drive/MyDrive/cse 291 datamining challenge/dm_files/my_model2")
    else:
      if  valid_accuracy>max(valid_accuracy_list):
        torch.save(model.state_dict(), "/content/drive/MyDrive/cse 291 datamining challenge/dm_files/my_model2")
    if epoch>=10:
      if valid_accuracy<min(valid_accuracy_list[-5:-1]):
        break
    valid_accuracy_list.append(valid_accuracy)
    valid_lost_epoch.append(np.mean(loss_list))
    print("epoch: "+str(epoch))
    print("train loss: "+str(train_lost_epoch[-1]))
    print("valid loss: "+str(valid_lost_epoch[-1]))
    print("valid accuracy: "+str(valid_accuracy))
    print("——————————————————————————————————————————————")


epoch:   0%|          | 0/300 [00:00<?, ?it/s]

trainset:   0%|          | 0/575 [00:00<?, ?it/s]

validset:   0%|          | 0/247 [00:00<?, ?it/s]

epoch: 0
train loss: 1.3487651282808055
valid loss: 1.3335458197091754
valid accuracy: 0.5664300202839757
——————————————————————————————————————————————


trainset:   0%|          | 0/575 [00:00<?, ?it/s]

...

trainset:   0%|          | 0/575 [00:00<?, ?it/s]

validset:   0%|          | 0/247 [00:00<?, ?it/s]

epoch: 18
train loss: 1.000765328925589
valid loss: 1.1080916579435711
valid accuracy: 0.6315922920892495
——————————————————————————————————————————————


trainset:   0%|          | 0/575 [00:00<?, ?it/s]

validset:   0%|          | 0/247 [00:00<?, ?it/s]

# fine-tuning last few layer

In [ ]:
model.load_state_dict(torch.load("/content/drive/MyDrive/cse 291 datamining challenge/dm_files/my_model2"))

<All keys matched successfully>

In [ ]:
for name, param in model.bert.named_parameters():
  if not name.startswith("encoder.layer.11"):
    param.requires_grad = False

In [ ]:
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5
                  eps = 1e-8, # args.adam_epsilon  - default is 1e-8
                  weight_decay=0.001,
                  correct_bias=True
                )
epochs = 300

In [ ]:
train_lost_epoch=[]
valid_lost_epoch=[]
valid_accuracy_list=[]
for epoch in tqdm(range(epochs),desc='epoch'):
    model.train()
    loss_list=[]
    i=0
    for batch in tqdm(train_dataloader,desc='trainset',leave=False):
        model.zero_grad()
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        target=torch.tensor(y_train[i:i+bat_size]).to(device)
        i+=bat_size
        outputs = model(input_ids,
                            token_type_ids,
                            attention_mask,
                            labels=target)

        loss = outputs.loss
        logits = outputs.logits
        loss.backward()
        optimizer.step()
        loss_list.append(loss.item())
    train_lost_epoch.append(np.mean(loss_list))
    model.eval()
    loss_list=[]
    i=0
    num_correct=0
    for batch in tqdm(valid_dataloader,desc='validset',leave=False):
      with torch.no_grad():
        input_ids = batch['input_ids'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        target=torch.tensor(y_valid[i:i+bat_size]).to(device)
        i+=bat_size
        outputs = model(input_ids,
                            token_type_ids,
                            attention_mask,
                            labels=target)
        loss = outputs.loss
        logits = outputs.logits
        loss_list.append(loss.item())
        pred_flat = np.argmax(logits.detach().cpu().numpy(), axis=1).flatten()
        labels_flat = target.to('cpu').numpy().flatten()
        num_correct += np.sum(pred_flat == labels_flat)
    valid_accuracy=num_correct/len(y_valid)
    if  epoch==0:
      torch.save(model.state_dict(), "/content/drive/MyDrive/cse 291 datamining challenge/dm_files/my_model3")
    else:
      if  valid_accuracy>max(valid_accuracy_list):
        torch.save(model.state_dict(), "/content/drive/MyDrive/cse 291 datamining challenge/dm_files/my_model3")
    if epoch>=10:
      if valid_accuracy<min(valid_accuracy_list[-5:-1]):
        break
    valid_accuracy_list.append(valid_accuracy)
    valid_lost_epoch.append(np.mean(loss_list))
    print("epoch: "+str(epoch))
    print("train loss: "+str(train_lost_epoch[-1]))
    print("valid loss: "+str(valid_lost_epoch[-1]))
    print("valid accuracy: "+str(valid_accuracy))
    print("——————————————————————————————————————————————")


epoch:   0%|          | 0/300 [00:00<?, ?it/s]

trainset:   0%|          | 0/575 [00:00<?, ?it/s]

validset:   0%|          | 0/247 [00:00<?, ?it/s]

epoch: 0
train loss: 1.012233811824218
valid loss: 1.077262064585319
valid accuracy: 0.6435091277890467
——————————————————————————————————————————————


trainset:   0%|          | 0/575 [00:00<?, ?it/s]

...

trainset:   0%|          | 0/575 [00:00<?, ?it/s]

validset:   0%|          | 0/247 [00:00<?, ?it/s]

epoch: 24
train loss: 0.7890201071811759
valid loss: 0.945244399219872
valid accuracy: 0.6906693711967545
——————————————————————————————————————————————


trainset:   0%|          | 0/575 [00:00<?, ?it/s]

validset:   0%|          | 0/247 [00:00<?, ?it/s]

#K fold traning:

In [ ]:
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels = len(label_list),
    output_attentions = False,
    output_hidden_states = False,
)
model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
model.load_state_dict(torch.load("/content/drive/MyDrive/cse 291 datamining challenge/dm_files/my_model4"))

<All keys matched successfully>

In [ ]:
optimizer = AdamW(model.parameters(),
                  lr = 3e-7, # args.learning_rate - default is 5e-5
                  eps = 1e-8, # args.adam_epsilon  - default is 1e-8
                  weight_decay=0.01,
                  correct_bias=True
                )
epochs = 15

In [ ]:
pip install nlpaug

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 10.0 MB/s eta 0:00:00


In [ ]:
import nlpaug.augmenter.word as naw
aug1 = naw.ContextualWordEmbsAug(
    model_path='bert-base-uncased', action="insert")
aug2 = naw.ContextualWordEmbsAug(
    model_path='bert-base-uncased', action="substitute")

In [ ]:
X_origin=list(df_train['train_text'])
y_origin=list(df_train['labels'])
# X_aug=[]
# y_aug=[]
# for i in tqdm(range(int(len(X_origin)/10))):
#   X_aug.append(aug1.augment(X_origin[i]))
#   y_aug.append(y_origin[i])
# X_origin.extend(X_aug)
# y_origin.extend(y_aug)

In [ ]:
for i in range(len(X_origin)):
  X_origin[i]=str(X_origin[i])

In [ ]:
from torch.utils.data import Dataset
class MyDataset(Dataset):
    def __init__(self, data_dict):
        self.data = data_dict
        self.keys = list(data_dict.keys())

    def __len__(self):
        return len(self.keys)

    def __getitem__(self, index):
        dic={}
        for key in self.keys:
          dic[key]=torch.Tensor(self.data[key][index])
        return dic
    def __len__(self):
        return len(self.data[self.keys[0]])

In [ ]:
from sklearn.model_selection import KFold
X_train_k=tokenizer(list(X_origin),truncation=True, padding=True, max_length=512,return_tensors='pt')
y_train_k=y_origin

In [ ]:
kf = KFold(n_splits=5)
train_lost_epoch=[]
valid_lost_epoch=[]
valid_accuracy_list=[]
for epoch in tqdm(range(epochs),desc='epoch'):
  fold_trainloss=[]
  fold_validloss=[]
  fold_accuracy=[]
  for t, (train_index, test_index) in tqdm(enumerate(kf.split(X_train_k["input_ids"])),leave=False):
    temp_train_k={"input_ids":X_train_k['input_ids'][train_index],"token_type_ids":X_train_k['token_type_ids'][train_index],\
                  "attention_mask":X_train_k['attention_mask'][train_index]}
    temp_valid_k={"input_ids":X_train_k['input_ids'][test_index],"token_type_ids":X_train_k['token_type_ids'][test_index],\
                  "attention_mask":X_train_k['attention_mask'][test_index]}
    train_k_dataloader = DataLoader(MyDataset(temp_train_k) , batch_size=bat_size)
    valid_k_dataloader = DataLoader(MyDataset(temp_valid_k), batch_size=bat_size)
    model.train()
    loss_list=[]
    i=0
    for batch in tqdm(train_k_dataloader,desc='trainset',leave=False):
        model.zero_grad()
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        target=torch.tensor([y_train_k[index] \
                             for index in train_index[i:min(len(train_index),i+bat_size)]])\
                             .to(device)
        i+=bat_size
        outputs = model(input_ids,
                            token_type_ids,
                            attention_mask,
                            labels=target)
        loss = outputs.loss
        logits = outputs.logits
        loss.backward()
        optimizer.step()
        loss_list.append(loss.item())
    fold_trainloss.append(np.mean(loss_list))

    model.eval()
    loss_list=[]
    i=0
    num_correct=0
    for batch in tqdm(valid_k_dataloader,desc='validset',leave=False):
      with torch.no_grad():
        input_ids = batch['input_ids'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        target=torch.tensor([y_train_k[index] \
                             for index in test_index[i:min(len(test_index),i+bat_size)]])\
                             .to(device)
        i+=bat_size
        outputs = model(input_ids,
                            token_type_ids,
                            attention_mask,
                            labels=target)
        loss = outputs.loss
        logits = outputs.logits
        loss_list.append(loss.item())
        pred_flat = np.argmax(logits.detach().cpu().numpy(), axis=1).flatten()
        labels_flat = target.to('cpu').numpy().flatten()
        num_correct += np.sum(pred_flat == labels_flat)
    fold_validloss.append(np.mean(loss_list))
    fold_accuracy.append(num_correct/len(temp_valid_k['input_ids']))
    print("fold: "+str(t))
    print("fold train loss: "+str(fold_trainloss[-1]))
    print("fold valid loss: "+str(fold_validloss[-1]))
    print("fold valid accuracy: "+str(fold_accuracy))
    print("#######################################################")
  valid_accuracy=np.mean(fold_accuracy)
  if  epoch==0:
    torch.save(model.state_dict(), "/content/drive/MyDrive/cse 291 datamining challenge/dm_files/my_model5")
  else:
    if  valid_accuracy>max(valid_accuracy_list):
      torch.save(model.state_dict(), "/content/drive/MyDrive/cse 291 datamining challenge/dm_files/my_model5")
  if epoch>=5:
    if valid_accuracy<min(valid_accuracy_list[-5:-1]):
      break

  valid_accuracy_list.append(np.mean(fold_accuracy))
  train_lost_epoch.append(np.mean(fold_validloss))
  valid_lost_epoch.append(np.mean(fold_validloss))
  print("epoch: "+str(epoch))
  print("train loss: "+str(fold_trainloss[-1]))
  print("valid loss: "+str(valid_lost_epoch[-1]))
  print("valid accuracy: "+str(valid_accuracy))
  print("——————————————————————————————————————————————")


epoch:   0%|          | 0/15 [00:00<?, ?it/s]

0it [00:00, ?it/s]

trainset:   0%|          | 0/658 [00:00<?, ?it/s]

validset:   0%|          | 0/165 [00:00<?, ?it/s]

fold: 0
fold train loss: 0.8361596591363019
fold valid loss: 0.8705162057370851
fold valid accuracy: [0.699885888170407]
#######################################################


...

trainset:   0%|          | 0/658 [00:00<?, ?it/s]

validset:   0%|          | 0/165 [00:00<?, ?it/s]

fold: 0
fold train loss: 0.96510559284216
fold valid loss: 0.9933194772763686
fold valid accuracy: [0.6432103461392165]
#######################################################


trainset:   0%|          | 0/658 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

#lower learning rate

In [ ]:
model.load_state_dict(torch.load("/content/drive/MyDrive/cse 291 datamining challenge/dm_files/my_model5"))
X_train, X_valid, y_train, y_valid = train_test_split(df_train['train_text'], df_train['labels'], test_size=0.3, random_state=52)
X_valid=tokenizer(list(X_valid),truncation=True, padding=True, max_length=512,return_tensors='pt')
y_valid=list(y_valid)
valid_dataloader = DataLoader(MyData(X_valid) , batch_size=bat_size)
i=0
for batch in tqdm(valid_dataloader,desc='validset',leave=False):
  with torch.no_grad():
    input_ids = batch['input_ids'].to(device)
    token_type_ids = batch['token_type_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    target=torch.tensor(y_valid[i:i+bat_size]).to(device)
    i+=bat_size
    outputs = model(input_ids,
                        token_type_ids,
                        attention_mask,
                        labels=target)
    loss = outputs.loss
    logits = outputs.logits
    loss_list.append(loss.item())
    pred_flat = np.argmax(logits.detach().cpu().numpy(), axis=1).flatten()
    labels_flat = target.to('cpu').numpy().flatten()
    num_correct += np.sum(pred_flat == labels_flat)
valid_accuracy=num_correct/len(y_valid)

validset:   0%|          | 0/247 [00:00<?, ?it/s]

In [ ]:
model.load_state_dict(torch.load("/content/drive/MyDrive/cse 291 datamining challenge/dm_files/my_model5"))
optimizer = AdamW(model.parameters(),
                  lr = 2e-8, # args.learning_rate - default is 5e-5
                  eps = 1e-8, # args.adam_epsilon  - default is 1e-8
                  weight_decay=0.01,
                  correct_bias=True
                )
epochs = 15

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
kf = KFold(n_splits=5)
train_lost_epoch=[]
valid_lost_epoch=[]
valid_accuracy_list=[]
for epoch in tqdm(range(epochs),desc='epoch'):
  fold_trainloss=[]
  fold_validloss=[]
  fold_accuracy=[]
  for t, (train_index, test_index) in tqdm(enumerate(kf.split(X_train_k["input_ids"])),leave=False):
    temp_train_k={"input_ids":X_train_k['input_ids'][train_index],"token_type_ids":X_train_k['token_type_ids'][train_index],\
                  "attention_mask":X_train_k['attention_mask'][train_index]}
    temp_valid_k={"input_ids":X_train_k['input_ids'][test_index],"token_type_ids":X_train_k['token_type_ids'][test_index],\
                  "attention_mask":X_train_k['attention_mask'][test_index]}
    train_k_dataloader = DataLoader(MyDataset(temp_train_k) , batch_size=bat_size)
    valid_k_dataloader = DataLoader(MyDataset(temp_valid_k), batch_size=bat_size)
    model.train()
    loss_list=[]
    i=0
    for batch in tqdm(train_k_dataloader,desc='trainset',leave=False):
        model.zero_grad()
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        target=torch.tensor([y_train_k[index] \
                             for index in train_index[i:min(len(train_index),i+bat_size)]])\
                             .to(device)
        i+=bat_size
        outputs = model(input_ids,
                            token_type_ids,
                            attention_mask,
                            labels=target)
        loss = outputs.loss
        logits = outputs.logits
        loss.backward()
        optimizer.step()
        loss_list.append(loss.item())
    fold_trainloss.append(np.mean(loss_list))

    model.eval()
    loss_list=[]
    i=0
    num_correct=0
    for batch in tqdm(valid_k_dataloader,desc='validset',leave=False):
      with torch.no_grad():
        input_ids = batch['input_ids'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        target=torch.tensor([y_train_k[index] \
                             for index in test_index[i:min(len(test_index),i+bat_size)]])\
                             .to(device)
        i+=bat_size
        outputs = model(input_ids,
                            token_type_ids,
                            attention_mask,
                            labels=target)
        loss = outputs.loss
        logits = outputs.logits
        loss_list.append(loss.item())
        pred_flat = np.argmax(logits.detach().cpu().numpy(), axis=1).flatten()
        labels_flat = target.to('cpu').numpy().flatten()
        num_correct += np.sum(pred_flat == labels_flat)
    fold_validloss.append(np.mean(loss_list))
    fold_accuracy.append(num_correct/len(temp_valid_k['input_ids']))
    print("fold: "+str(t))
    print("fold train loss: "+str(fold_trainloss[-1]))
    print("fold valid loss: "+str(fold_validloss[-1]))
    print("fold valid accuracy: "+str(fold_accuracy))
    print("#######################################################")
  valid_accuracy=np.mean(fold_accuracy)
  if  epoch==0:
    torch.save(model.state_dict(), "/content/drive/MyDrive/cse 291 datamining challenge/dm_files/my_model6")
  else:
    if  valid_accuracy>max(valid_accuracy_list):
      torch.save(model.state_dict(), "/content/drive/MyDrive/cse 291 datamining challenge/dm_files/my_model6")
  if epoch>=3:
    if valid_accuracy<min(valid_accuracy_list[-3:-1]):
      break

  valid_accuracy_list.append(np.mean(fold_accuracy))
  train_lost_epoch.append(np.mean(fold_trainloss))
  valid_lost_epoch.append(np.mean(fold_accuracy))
  print("epoch: "+str(epoch))
  print("train loss: "+str(train_lost_epoch[-1]))
  print("valid loss: "+str(valid_lost_epoch[-1]))
  print("valid accuracy: "+str(valid_accuracy))
  print("——————————————————————————————————————————————")


epoch:   0%|          | 0/15 [00:00<?, ?it/s]

0it [00:00, ?it/s]

trainset:   0%|          | 0/658 [00:00<?, ?it/s]

validset:   0%|          | 0/165 [00:00<?, ?it/s]

fold: 0
fold train loss: 0.8150914599467918
fold valid loss: 0.9370830539501075
fold valid accuracy: [0.6770635222518068]
#######################################################


trainset:   0%|          | 0/658 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
fold_accuracy

358

In [ ]:
model.load_state_dict(torch.load("/content/drive/MyDrive/cse 291 datamining challenge/dm_files/my_model5"))
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5
                  eps = 1e-8, # args.adam_epsilon  - default is 1e-8
                  weight_decay=0.1,
                  correct_bias=True
                )
for name, param in model.bert.named_parameters():
  if not name.startswith("encoder.layer.11"):
    param.requires_grad = False
epochs = 30

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
kf = KFold(n_splits=5)
train_lost_epoch=[]
valid_lost_epoch=[]
valid_accuracy_list=[]
for epoch in tqdm(range(epochs),desc='epoch'):
  fold_trainloss=[]
  fold_validloss=[]
  fold_accuracy=[]
  for t, (train_index, test_index) in tqdm(enumerate(kf.split(X_train_k["input_ids"])),leave=False):
    temp_train_k={"input_ids":X_train_k['input_ids'][train_index],"token_type_ids":X_train_k['token_type_ids'][train_index],\
                  "attention_mask":X_train_k['attention_mask'][train_index]}
    temp_valid_k={"input_ids":X_train_k['input_ids'][test_index],"token_type_ids":X_train_k['token_type_ids'][test_index],\
                  "attention_mask":X_train_k['attention_mask'][test_index]}
    train_k_dataloader = DataLoader(MyDataset(temp_train_k) , batch_size=bat_size)
    valid_k_dataloader = DataLoader(MyDataset(temp_valid_k), batch_size=bat_size)
    model.train()
    loss_list=[]
    i=0
    for batch in tqdm(train_k_dataloader,desc='trainset',leave=False):
        model.zero_grad()
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        target=torch.tensor([y_train_k[index] \
                             for index in train_index[i:min(len(train_index),i+bat_size)]])\
                             .to(device)
        i+=bat_size
        outputs = model(input_ids,
                            token_type_ids,
                            attention_mask,
                            labels=target)
        loss = outputs.loss
        logits = outputs.logits
        loss.backward()
        optimizer.step()
        loss_list.append(loss.item())
    fold_trainloss.append(np.mean(loss_list))

    model.eval()
    loss_list=[]
    i=0
    num_correct=0
    for batch in tqdm(valid_k_dataloader,desc='validset',leave=False):
      with torch.no_grad():
        input_ids = batch['input_ids'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        target=torch.tensor([y_train_k[index] \
                             for index in test_index[i:min(len(test_index),i+bat_size)]])\
                             .to(device)
        i+=bat_size
        outputs = model(input_ids,
                            token_type_ids,
                            attention_mask,
                            labels=target)
        loss = outputs.loss
        logits = outputs.logits
        loss_list.append(loss.item())
        pred_flat = np.argmax(logits.detach().cpu().numpy(), axis=1).flatten()
        labels_flat = target.to('cpu').numpy().flatten()
        num_correct += np.sum(pred_flat == labels_flat)
    fold_validloss.append(np.mean(loss_list))
    fold_accuracy.append(num_correct/len(temp_valid_k['input_ids']))
    print("fold: "+str(t))
    print("fold train loss: "+str(fold_trainloss[-1]))
    print("fold valid loss: "+str(fold_validloss[-1]))
    print("fold valid accuracy: "+str(fold_accuracy))
    print("#######################################################")
  valid_accuracy=np.mean(fold_accuracy)
  if  epoch==0:
    torch.save(model.state_dict(), "/content/drive/MyDrive/cse 291 datamining challenge/dm_files/my_model6")
  else:
    if  valid_accuracy>max(valid_accuracy_list):
      torch.save(model.state_dict(), "/content/drive/MyDrive/cse 291 datamining challenge/dm_files/my_model6")
  if epoch>=5:
    if valid_accuracy<min(valid_accuracy_list[-5:-1]):
      break

  valid_accuracy_list.append(np.mean(fold_accuracy))
  train_lost_epoch.append(np.mean(fold_trainloss))
  valid_lost_epoch.append(np.mean(fold_validloss))
  print("epoch: "+str(epoch))
  print("train loss: "+str(train_lost_epoch[-1]))
  print("valid loss: "+str(valid_lost_epoch[-1]))
  print("valid accuracy: "+str(valid_accuracy))
  print("——————————————————————————————————————————————")


epoch:   0%|          | 0/30 [00:00<?, ?it/s]

0it [00:00, ?it/s]

trainset:   0%|          | 0/658 [00:00<?, ?it/s]

validset:   0%|          | 0/165 [00:00<?, ?it/s]

fold: 0
fold train loss: 0.8380760219072679
fold valid loss: 0.9498315343351075
fold valid accuracy: [0.6728794218333968]
#######################################################


...

trainset:   0%|          | 0/658 [00:00<?, ?it/s]

validset:   0%|          | 0/165 [00:00<?, ?it/s]

fold: 0
fold train loss: 0.8119528405481559
fold valid loss: 0.9462787272352161
fold valid accuracy: [0.6618486116394067]
#######################################################


trainset:   0%|          | 0/658 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
test_dataloader = DataLoader(MyData(X_train_k) , batch_size=bat_size)
i=0
for batch in tqdm(test_dataloader,desc='validset',leave=False):
  with torch.no_grad():
    input_ids = batch['input_ids'].to(device)
    token_type_ids = batch['token_type_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    target=torch.tensor(y_train_k[i:i+bat_size]).to(device)
    i+=bat_size
    outputs = model(input_ids,
                        token_type_ids,
                        attention_mask,
                        labels=target)
    logits = outputs.logits
    pred_flat = np.argmax(logits.detach().cpu().numpy(), axis=1).flatten()
    labels_flat = target.to('cpu').numpy().flatten()
    num_correct += np.sum(pred_flat == labels_flat)
valid_accuracy=num_correct/len(y_train_k)

validset:   0%|          | 0/822 [00:00<?, ?it/s]

In [ ]:
valid_accuracy

0.7957242848447961

In [ ]:
len(X_train_k['input_ids'])

13144

In [ ]:
len(y_train_k)

13144

In [ ]:
valid_accuracy

0.040398660986001214

# Make prediction

In [ ]:
model.load_state_dict(torch.load("/content/drive/MyDrive/cse 291 datamining challenge/dm_files/my_model2"))

<All keys matched successfully>

In [ ]:
df_test=preprocess(df_test)

0it [00:00, ?it/s]

In [ ]:
X_origin=list(df_test['train_text'])
X_test_k=tokenizer(list(X_origin),truncation=True, padding=True, max_length=512,return_tensors='pt')

In [ ]:
test_dataloader = DataLoader(MyData(X_test_k) , batch_size=bat_size)

In [ ]:
model.eval()
result=np.zeros((1,1))
for batch in tqdm(test_dataloader,desc='testset',leave=False):
  with torch.no_grad():
    temp_test_k={"input_ids":X_test_k['input_ids'][test_index],"token_type_ids":X_test_k['token_type_ids'][test_index],\
                  "attention_mask":X_test_k['attention_mask'][test_index]}
    test_k_dataloader = DataLoader(MyDataset(temp_test_k) , batch_size=bat_size)
    input_ids = batch['input_ids'].to(device)
    token_type_ids = batch['token_type_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    target=torch.tensor(y_valid[i:i+bat_size]).to(device)
    i+=bat_size
    outputs = model(input_ids,
                        token_type_ids,
                        attention_mask)
    logits = outputs.logits
    pred_flat = np.argmax(logits.detach().cpu().numpy(), axis=1).reshape(1,-1)
    result=np.concatenate((result, pred_flat),axis=1)

testset:   0%|          | 0/625 [00:00<?, ?it/s]

In [ ]:
result[0:5]

array([[0., 4., 9., ..., 0., 0., 2.]])

In [ ]:
result_final=list(result[0,1:])

In [ ]:
len(result_final)

10000

In [ ]:
result_final[0:5]

[4.0, 9.0, 3.0, 3.0, 5.0]

In [ ]:
dic = {"Id": [], "Predicted": []}
for i, pred in enumerate(result_final):
  dic["Id"].append(i)
  dic["Predicted"].append(label_list[int(pred)])
dic_df = pd.DataFrame.from_dict(dic)
dic_df.to_csv(data_path + "predicted_mymodel.csv", index=False)

In [ ]:
print(dic)

{'Id': [], 'Predicted': []}


# No Preprocess

In [ ]:
bat_size=32

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(df_train['review'], df_train['labels'], test_size=0.3, random_state=52)
X_train=tokenizer(list(X_train),truncation=True, padding=True, max_length=300,return_tensors='pt')
y_train=list(y_train)
X_valid=tokenizer(list(X_valid),truncation=True, padding=True, max_length=300,return_tensors='pt')
y_valid=list(y_valid)
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels = len(label_list),
    output_attentions = False,
    output_hidden_states = False,
)
model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
train_dataloader = DataLoader(MyData(X_train) , batch_size=bat_size)
valid_dataloader = DataLoader(MyData(X_valid) , batch_size=bat_size)

In [ ]:
optimizer = AdamW(model.parameters(),
                  lr = 5e-6, # args.learning_rate - default is 5e-5
                  eps = 1e-8, # args.adam_epsilon  - default is 1e-8
                  weight_decay=0.001,
                  correct_bias=True
                )
epochs = 300

In [ ]:
train_lost_epoch=[]
valid_lost_epoch=[]
valid_accuracy_list=[]
for epoch in tqdm(range(epochs),desc='epoch'):
    model.train()
    loss_list=[]
    i=0
    for batch in tqdm(train_dataloader,desc='trainset',leave=False):
        model.zero_grad()
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        target=torch.tensor(y_train[i:i+bat_size]).to(device)
        i+=bat_size
        outputs = model(input_ids,
                            token_type_ids,
                            attention_mask,
                            labels=target)

        loss = outputs.loss
        logits = outputs.logits
        loss.backward()
        optimizer.step()
        loss_list.append(loss.item())
    train_lost_epoch.append(np.mean(loss_list))
    model.eval()
    loss_list=[]
    i=0
    num_correct=0
    for batch in tqdm(valid_dataloader,desc='validset',leave=False):
      with torch.no_grad():
        input_ids = batch['input_ids'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        target=torch.tensor(y_valid[i:i+bat_size]).to(device)
        i+=bat_size
        outputs = model(input_ids,
                            token_type_ids,
                            attention_mask,
                            labels=target)
        loss = outputs.loss
        logits = outputs.logits
        loss_list.append(loss.item())
        pred_flat = np.argmax(logits.detach().cpu().numpy(), axis=1).flatten()
        labels_flat = target.to('cpu').numpy().flatten()
        num_correct += np.sum(pred_flat == labels_flat)
    valid_accuracy=num_correct/len(y_valid)
    if  epoch==0:
      torch.save(model.state_dict(), "/content/drive/MyDrive/cse 291 datamining challenge/dm_files/my_model_nopre1")
    else:
      if  valid_accuracy>max(valid_accuracy_list):
        torch.save(model.state_dict(), "/content/drive/MyDrive/cse 291 datamining challenge/dm_files/my_model_nopre1")
    if epoch>=10:
      if valid_accuracy<min(valid_accuracy_list[-5:-1]):
        break
    valid_accuracy_list.append(valid_accuracy)
    valid_lost_epoch.append(np.mean(loss_list))
    print("epoch: "+str(epoch))
    print("train loss: "+str(train_lost_epoch[-1]))
    print("valid loss: "+str(valid_lost_epoch[-1]))
    print("valid accuracy: "+str(valid_accuracy))
    print("——————————————————————————————————————————————")


epoch:   0%|          | 0/300 [00:00<?, ?it/s]

trainset:   0%|          | 0/288 [00:00<?, ?it/s]

validset:   0%|          | 0/124 [00:00<?, ?it/s]

epoch: 0
train loss: 2.0788393057882786
valid loss: 1.8707598313208549
valid accuracy: 0.3564908722109533
——————————————————————————————————————————————


trainset:   0%|          | 0/288 [00:00<?, ?it/s]

...

trainset:   0%|          | 0/288 [00:00<?, ?it/s]

validset:   0%|          | 0/124 [00:00<?, ?it/s]

epoch: 12
train loss: 1.249829064640734
valid loss: 1.252128446775098
valid accuracy: 0.5788539553752535
——————————————————————————————————————————————


trainset:   0%|          | 0/288 [00:00<?, ?it/s]

validset:   0%|          | 0/124 [00:00<?, ?it/s]